In [ ]:
# """fraction of expected number of event over number of event assuming no oscilation (294.7 )
# """
# import numpy as np

# from framework_pkg.framework import FrameWork
# from framework_pkg.survival_probablity import MSW

# frame      = FrameWork(resolution_correction=False, masked_val=0.1)
# spectrum   = frame.spectrum_nu
# t_recoil   = np.concatenate((np.logspace(-5,-2,30), frame.energy_recoil))
# electron_cross_section = frame._compute_cross_section(frame.energy_nu,t_recoil,1)
# moun_cross_section     = frame._compute_cross_section(frame.energy_nu,t_recoil,-1)
# SNO_norm = 1e-4 * frame.norm


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from framework_pkg.total_flux import SuperKFlux

plt.rcParams['text.usetex']=True
plt.style.use('classic')

In [2]:
param = {'SinT12': 0.308, 'M12': 6.9e-5}

MSWSpectrum   = SuperKFlux()
spectrum_data = MSWSpectrum.modulation_data
energy_obs    = MSWSpectrum.energy_obs

flux_msw = MSWSpectrum.__getitem__(param, name="MSW")
flux_pseudodirac = MSWSpectrum.__getitem__(param, name="PseudoDirac")

In [3]:
print(flux_msw/5.25)
print()
print(flux_pseudodirac)

[0.43930183]

[1.56588905]


In [ ]:
        # if do_binning:
        #     self.data = self._bin_data(self.data)

In [2]:
"""expected number of event per day assuming no oscilation (294.7 per day per kilo ton )"""

integral_electron_mc = np.zeros(len(t_recoil))
k = 0
for i in range (len(t_recoil)):
    if t_recoil[i]<frame.energy_recoil[0]:
        integral_electron_mc[i] = np.trapz(frame.spectrum_nu * electron_cross_section[i,:], frame.energy_nu)
    else:
        integral_electron_mc[i] = np.trapz(frame.spectrum_nu[k:] * electron_cross_section[i,k:], frame.energy_nu[k:])
        k = k+1

total_volume  = 32.5
total_days    = 1
target_number = frame.target_number
mc_expected_event = total_days * total_volume * target_number * SNO_norm * np.trapz(integral_electron_mc,t_recoil)

print(mc_expected_event)

294.63483944691194


In [3]:
""" expected number of event per day assuming MSW survival probability """

frame.param.update({'SinT12': 0.319, 'M12': 7.51e-5})
survival_probablity = MSW(frame.param,frame.energy_nu)
integral_electron = np.zeros(len(t_recoil))
integral_moun  = np.zeros(len(t_recoil))
k = 0
for i in range (len(t_recoil)):
    if t_recoil[i] < frame.energy_recoil[0]:
        integral_electron[i] = np.trapz(frame.spectrum_nu * electron_cross_section[i,:] * survival_probablity,
                                          frame.energy_nu)
        integral_moun[i] = np.trapz(frame.spectrum_nu * moun_cross_section[i,:] * (1-survival_probablity),
                                          frame.energy_nu)
    else:
        integral_electron[i]= np.trapz(frame.spectrum_nu[k:] * electron_cross_section[i,k:]* survival_probablity[k:],
                                       frame.energy_nu[k:])
        integral_moun[i] = np.trapz(frame.spectrum_nu[k:] * moun_cross_section[i,k:] * (1-survival_probablity[k:]),
                                              frame.energy_nu[k:])
        k = k+1
suk_expected_event_e = total_days * total_volume * target_number * SNO_norm * np.trapz(integral_electron, t_recoil)
suk_expected_event_m = total_days * total_volume * target_number * SNO_norm * np.trapz(integral_moun, t_recoil)

print(suk_expected_event_e , suk_expected_event_m)

105.59138020099317 33.63970070107935


In [4]:
print((suk_expected_event_e+suk_expected_event_m)/mc_expected_event)

0.47255470929180293


In [ ]:
t12, mum2, m12 = param_update

self.param.update({'SinT12': t12, 'mum2': mum2, 'M12': m12})

survival_prob, sterile_prob = PseudoDirac(self.param, self.distance, self.energy_nu)

r = np.zeros((len(self.distance),self.energy_recoil.shape[0]))

for z,ts in enumerate(self.energy_recoil):
    if (len(self.energy_nu) - len(self.energy_nu[z:]))/len(self.energy_nu) >= 0.8 :
        r[:,z] = np.trapz(self.spectrum_nu * (self.cs_electron[z,:] * survival_prob + self.cs_muon[z,:] * (1 - survival_prob - sterile_prob)), self.energy_nu, axis=1) - np.trapz(self.spectrum_nu[:z] * (self.cs_electron[z,:z] * survival_prob[:,:z] + self.cs_muon[z,:z] * (1 - survival_prob[:,:z] - sterile_prob[:,:z])), self.energy_nu[:z], axis=1)
    else:
        r[:,z] = np.trapz(self.spectrum_nu[z:] * (self.cs_electron[z,z:] * survival_prob[:,z:] + self.cs_muon[z,z:] * (1 - survival_prob[:,z:] - sterile_prob[:,z:])), self.energy_nu[z:],axis=1)
    
if self.resolution_correction:
    self.flux_fraction_prediction = (self.norm/self.borom_unoscilated_total) * np.trapz( r * self.resp_func, self.energy_recoil, axis=1)/self.distance**2
else:
    self.flux_fraction_prediction = (self.norm/self.borom_unoscilated_total) * np.trapz( r , self.energy_recoil, axis=1)/self.distance**2
return self.flux_fraction_prediction